### this segmentation is a sub-segmentation of the original segmentations provided by the dataset

1) create a dataset of syllable waveforms for each isolation segment

2) get segmented times for each syllable in that segment

3) create a new dataframe with the isolated syllables

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.autonotebook import tqdm
from joblib import Parallel, delayed
import umap
import pandas as pd
from datetime import datetime


/tmp/ipykernel_311972/2400570142.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-04-25 22:55:40.197875: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 22:55:42.274083: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-25 22:55:42.274169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [3]:
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir

PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis


In [4]:
DATASET_ID = 'fruitbat'

In [5]:
# create a unique datetime identifier for the files output by this notebook
DT_ID = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
DT_ID

'2023-04-25_22-55-44'

In [6]:
from avgn.utils.hparams import HParams
from avgn.dataset import DataSet

In [7]:
from avgn.signalprocessing.create_spectrogram_dataset import prepare_wav, create_label_df, get_row_audio

### create dataset

In [8]:
hparams = HParams(
    num_mel_bins = 32,
    mel_lower_edge_hertz=500,
    mel_upper_edge_hertz=120000,
    butter_lowcut = 500,
    butter_highcut = 120000,
    ref_level_db = 20,
    min_level_db = -60,
    mask_spec = True,
    win_length_ms = 0.5,
    hop_length_ms = 0.05,
    mask_spec_kwargs = {"spec_thresh": 0.9, "offset": 1e-10},
    n_jobs = -1,
    verbosity=1,
    nex = -1
)

In [9]:
# create a dataset object
dataset = DataSet(DATASET_ID, hparams = hparams)

loading json:   0%|          | 0/87985 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 153 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 2128 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 57344 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 87985 out of 87985 | elapsed:    7.0s finished


getting unique individuals:   0%|          | 0/87985 [00:00<?, ?it/s]

2023-04-25 22:55:53.788377: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-04-25 22:55:53.788445: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-04-25 22:55:53.789417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# dataset.data_files = {i:dataset.data_files[i] for i in list(dataset.data_files.keys())[:5]}

In [ ]:
dataset.sample_json

In [ ]:
166697
dataset

In [ ]:
len(dataset.data_files)

#### Create dataset based upon JSON

In [10]:
from joblib import Parallel, delayed
n_jobs = 4; verbosity = 10

In [11]:
with Parallel(n_jobs=n_jobs, verbose=verbosity) as parallel:
    syllable_dfs = parallel(
        delayed(create_label_df)(
            dataset.data_files[key].data,
            hparams=dataset.hparams,
            labels_to_retain=["context"],
            unit="syllables",
            dict_features_to_retain = [],
            key = key,
        )
        for key in tqdm(dataset.data_files.keys())
    )
syllable_df = pd.concat(syllable_dfs)
len(syllable_df)

PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/

  0%|          | 0/87985 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1671s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done  25 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0106s.) Setting batch_size=4.
[Parallel(n_jobs=4)]: Done  44 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0145s.) Setting batch_size=8.
[Parallel(n_jobs=4)]: Batch computation too fast (0.0274s.) Setting batch_size=16.
[Parallel(n_jobs=4)]: Done  88 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0474s.) Setting batch_size=32.
[Parallel(n_jobs=4)]: Done 216 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0545s.) Setting batch_size=64.
[Pa

91079

PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis
PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis


In [ ]:
syllable_df[:3]

In [ ]:
syllable_df.context.unique()

In [ ]:
# get only isolation calls
#syllable_df = syllable_df[syllable_df.context == "Isolation"]

### get audio for dataset

In [ ]:
ls

In [ ]:
key = syllable_df.key.unique()[0];
get_row_audio(
            syllable_df[syllable_df.key == key], 
            dataset.data_files[key].data['wav_loc'], 
            dataset.hparams
        )

In [ ]:
with Parallel(n_jobs=n_jobs, verbose=verbosity) as parallel:
    syllable_dfs = parallel(
        delayed(get_row_audio)(
            syllable_df[syllable_df.key == key], 
            dataset.data_files[key].data['wav_loc'], 
            dataset.hparams
        )
        for key in tqdm(syllable_df.key.unique())
    )
syllable_df = pd.concat(syllable_dfs)
len(syllable_df)

In [ ]:
df_mask  = np.array([len(i) > 0 for i in tqdm(syllable_df.audio.values)])

In [ ]:
syllable_df = syllable_df[np.array(df_mask)]

In [ ]:
syllable_df[:3]

In [ ]:
sylls = syllable_df.audio.values

In [ ]:
nrows = 5
ncols = 10
zoom = 2
fig, axs = plt.subplots(ncols=ncols, nrows = nrows,figsize = (ncols*zoom, nrows+zoom/1.5))
for i, syll in tqdm(enumerate(sylls), total = nrows*ncols):
    ax = axs.flatten()[i]
    ax.plot(syll)
    if i == nrows*ncols -1:
        break

In [ ]:
syllable_df['audio'] = [i/np.max(i) for i in tqdm(syllable_df.audio.values)]

### segment isolation calls

In [ ]:
from vocalseg.dynamic_thresholding import dynamic_threshold_segmentation
from vocalseg.dynamic_thresholding import plot_segmented_spec, plot_segmentations
from vocalseg.utils import butter_bandpass_filter, spectrogram, int16tofloat32, plot_spec


In [ ]:
len(syllable_df)

In [ ]:
syllable_df[:3]

In [ ]:
pd.DataFrame({i: [np.sum(i == syllable_df.indv)] for i in syllable_df.indv.unique()}).T.sort_values(by=0, ascending=False).T

In [ ]:
n_fft=1024
hop_length_ms=.5
win_length_ms=4
ref_level_db=20
pre=0.97
min_level_db=-30
silence_threshold = 0.1
min_silence_for_spec=0.1
max_vocal_for_spec=1.0,
min_syllable_length_s = 0.01
spectral_range = [5000, 60000]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

display_results=False
start_times = []
end_times = []
for idx, row in tqdm(syllable_df.iterrows(), total=len(syllable_df)):
    results = dynamic_threshold_segmentation(
        row.audio,
        row.rate,
        n_fft=n_fft,
        hop_length_ms=hop_length_ms,
        win_length_ms=win_length_ms,
        ref_level_db=ref_level_db,
        pre=pre,
        min_level_db=min_level_db,
        
        silence_threshold=silence_threshold,
        verbose=False,
        spectral_range=spectral_range,
        min_syllable_length_s=min_syllable_length_s,
                min_level_db_floor=20,

    )
    
    if results is None:
        if display_results:
            spec = spectrogram(
                row.audio,
                row.rate,
                n_fft=n_fft,
                hop_length_ms=hop_length_ms,
                win_length_ms=win_length_ms,
                ref_level_db=ref_level_db,
                pre=pre,
                min_level_db=min_level_db,
            )
            fig, ax = plt.subplots(figsize=(30,5))
            plot_spec(spec, fig, ax);
            plt.show()
        start_times.append(None)
        end_times.append(None)
        continue
    else:
        if display_results:
            plot_segmentations(
                results["spec"],
                vocal_envelope=results["vocal_envelope"],
                onsets=results["onsets"],
                offsets=results["offsets"],
                rate=row.rate,
                hop_length_ms=hop_length_ms,
                figsize=(30, 5),
                #spectral_range=spectral_range
            )
            plt.show()
            
    start_times.append(results['onsets'])
    end_times.append(results['offsets'])

In [ ]:
len((np.concatenate([i for i in start_times if i is not None])))

In [ ]:
results.keys()

In [ ]:
start_times[:3]

### add start and end times to json

In [ ]:
from avgn.utils.json import NoIndent, NoIndentEncoder

In [ ]:
syllable_df['start_times'] = start_times

In [ ]:
syllable_df['end_times'] = end_times

In [ ]:
syllable_df[:3]

In [ ]:
for idx, row in tqdm(syllable_df.iterrows(), total=len(syllable_df)):
    
    if row.start_times is None: 
        continue
    
    new_json = dataset.data_files[row.key].data.copy()
    new_json['indvs'][row.indv]['elements'] = {
        "start_times": NoIndent(list(row.start_times)),
        "end_times": NoIndent(list(row.end_times)),
    }
    json_out = (
        DATA_DIR
        / "processed"
        / (DATASET_ID + '_isolate_segmented')
        / DT_ID
        / "JSON"
        / (row.key + ".JSON")
    )
    
    json_txt = json.dumps(new_json, cls=NoIndentEncoder, indent=2)
    
    ensure_dir(json_out.as_posix())
    print(json_txt, file=open(json_out.as_posix(), "w"))
    #break